<a href="https://colab.research.google.com/github/sammsc/meeting-notes-ai/blob/main/temi2summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
import re
from pathlib import Path
import pandas as pd
from transformers import pipeline

     |████████████████████████████████| 3.1 MB 13.5 MB/s 
     |████████████████████████████████| 895 kB 43.5 MB/s 
     |████████████████████████████████| 596 kB 47.6 MB/s 
     |████████████████████████████████| 3.3 MB 22.3 MB/s 
     |████████████████████████████████| 61 kB 521 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
path_root = Path('/gdrive/My Drive/meeting_notes_ai')
file_txt = 'transcript_with_speaker.txt'
with open(Path(path_root / file_txt)) as fp:
    text_all = fp.readlines()

In [4]:
summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum")

Downloading:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [5]:
df_transcript = pd.DataFrame(columns=['speaker', 'time', 'text', 'summary'])
first_line = True

for line_idx, line_cur in enumerate(text_all):
    if len(line_cur.strip()) != 0:
        # matching speaker ID line
        m_res = re.search(r'^Speaker (?P<id>\d+) \((?P<time>\d+:\d+)\):\s+\Z', line_cur)
        if m_res:
            if first_line:
                first_line = False
            else:
                txt_summary = summarizer(txt_orig, max_length=90, min_length=25, do_sample=False)
                df_transcript = df_transcript.append(pd.Series(
                    [speaker_id, speaker_time, txt_orig, txt_summary],
                    index=['speaker', 'time', 'text', 'summary']),
                                               ignore_index=True)
            speaker_id = m_res.group('id')
            speaker_time = m_res.group('time')
            txt_orig = ''
        else:
            txt_orig += line_cur.rstrip('\n')

txt_summary = summarizer(txt_orig, max_length=90, min_length=25, do_sample=False)
df_transcript = df_transcript.append(pd.Series(
    [speaker_id, speaker_time, txt_orig, txt_summary],
    index=['speaker', 'time', 'text', 'summary']),
    ignore_index=True)


Your max_length is set to 90, but you input_length is only 24. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 90, but you input_length is only 58. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 90, but you input_length is only 46. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 90, but you input_length is only 74. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 90, but you input_length is only 73. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 90, but you input_length is only 71. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 90, but you input_length is only 39. You might consider decrea

In [6]:
df_transcript.to_csv(Path(path_root / 'df_transcript.csv'), index=False)
df_transcript.to_pickle(Path(path_root / 'df_transcript.pkl'))

In [7]:
df_transcript.head()

,speaker,time,text,summary
0,1,00:02,"And we have our weekly, uh, product marketing ...",[{'summary_text': 'Cormac and Cindy have login...
1,2,01:10,"The main thing? Well, there are some, so it de...",[{'summary_text': 'Forrester used to have an i...
2,3,02:26,I remember hearing a data and analysis offerin...,[{'summary_text': ' is a data and analysis off...
3,2,02:32,"Um, yeah, like technographics is the, the Fore...",[{'summary_text': 'The Forester gets much broa...
4,1,03:04,"Yeah, that's uh, that would be the general ide...",[{'summary_text': 'Some of the people on the c...
